#  <font color='orangered'> Текущий уровень потребительской лояльности, среди клиентов из России. </font> 

**Цель:**
Заказчик этого исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России. 

# <font color='orangered'> Оглавление </font> 

1. [Загрузка библиотек](#start1)
2. [Подключение к базе](#start2)
3. [Загрузка данных средствами SQL](#start3)
4. [Предобработка данных](#start4)

5. [Ссылка на дашборд](#superend")


## 1. Загрузка библиотек <a id="start"></a> 

In [9]:
import os
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

## 2. Подключение к базе <a id="start2"></a> 

In [10]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = r'C:\Users\NA321\Desktop\яндекс_проекты\telecomm_csi.db' 
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

## 3. Загрузка данных средствами SQL <a id="start3"></a> 

In [11]:

query = """
WITH 
u AS (SELECT *
       FROM user),

z AS (SELECT *
      FROM location),

s  AS (SELECT * 
     FROM age_segment),
     
n AS (SELECT * 
      FROM traffic_segment),

l AS (SELECT *
      FROM lifetime_segment)



SELECT  u.user_id,
        u.lt_day,
        (CASE 
            WHEN u.lt_day <= 365 THEN 'new' 
            ELSE 'old' 
            END )AS is_new,
        u.age,
        (CASE 
            WHEN u.gender_segment = 1 THEN 'women' 
            WHEN u.gender_segment = 0 THEN 'men'
            ELSE 'other' 
            END )AS gender_segment,
        u.os_name,
          
        u.cpe_type_name,
        z.country,
        z.city,
        SUBSTR (s.title, 3) AS age_segment,
        SUBSTR (n.title, 3) As traffic_segment,
        SUBSTR (l.title, 3) AS lifetime_segment,
        u.nps_score,
        (CASE
           WHEN u.nps_score < 7 THEN 'detractors'
           WHEN u.nps_score >= 7 AND u.nps_score < 9 THEN 'passives'
           WHEN u.nps_score >= 9 THEN 'promoters'
        END) AS nps_group
        
FROM  u
LEFT JOIN z ON z.location_id=u.location_id
LEFT JOIN s ON s.age_gr_id=u.age_gr_id
LEFT JOIN n ON n.tr_gr_id=u.tr_gr_id
LEFT JOIN l ON l.lt_gr_id=u.lt_gr_id;
"""



## 4. Предобработка данных <a id="start4"></a> 

In [12]:
df = pd.read_sql(query, engine)
df.head(5)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,old,45.0,women,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,promoters
1,A001WF,2344,old,53.0,men,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,promoters
2,A003Q7,467,old,57.0,men,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,promoters
3,A004TB,4190,old,44.0,women,IOS,SMARTPHONE,Россия,РостовнаДону,35-44,0.1-1,36+,10,promoters
4,A004XT,1163,old,24.0,men,ANDROID,SMARTPHONE,Россия,Рязань,16-24,5-10,36+,10,promoters


In [13]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

Получим информацию о данных, а также информацию о пропусках.

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           502493 non-null  object 
 1   lt_day            502493 non-null  int64  
 2   is_new            502493 non-null  object 
 3   age               501939 non-null  float64
 4   gender_segment    502493 non-null  object 
 5   os_name           502493 non-null  object 
 6   cpe_type_name     502493 non-null  object 
 7   country           502493 non-null  object 
 8   city              502493 non-null  object 
 9   age_segment       502493 non-null  object 
 10  traffic_segment   502493 non-null  object 
 11  lifetime_segment  502493 non-null  object 
 12  nps_score         502493 non-null  int64  
 13  nps_group         502493 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 53.7+ MB


In [15]:
df.isna().sum()

user_id               0
lt_day                0
is_new                0
age                 554
gender_segment        0
os_name               0
cpe_type_name         0
country               0
city                  0
age_segment           0
traffic_segment       0
lifetime_segment      0
nps_score             0
nps_group             0
dtype: int64

Данные содержат информацию о 502493 пользователей, и 554 пропуска в столбце с информацией о возрасте.

In [16]:
df.describe()

,lt_day,age,nps_score
count,502493.000000,501939.000000,502493.000000
mean,1868.841439,39.621946,7.508562
std,1683.701762,11.188249,3.020378
min,-21.000000,10.000000,1.000000
25%,533.000000,31.000000,5.000000
50%,1239.000000,38.000000,9.000000
75%,3064.000000,47.000000,10.000000
max,9162.000000,89.000000,10.000000


In [17]:
problem_df1 = df.loc[df['lt_day'] < 0]
problem_df1

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
7666,AEAC2R,-8,new,NaN,other,ANDROID,SMARTPHONE,Россия,Томск,n/a,15-20,36+,1,detractors
53997,CS0HF8,-2,new,NaN,other,ANDROID,SMARTPHONE,Россия,Ижевск,n/a,20-25,36+,10,promoters
71577,DORPT2,-4,new,NaN,other,ANDROID,SMARTPHONE,Россия,Москва,n/a,20-25,36+,4,detractors
100152,F5O3CG,-21,new,NaN,other,ANDROID,SMARTPHONE,Россия,Чита,n/a,35-40,36+,7,passives
228087,LSE939,-13,new,NaN,other,ANDROID,SMARTPHONE,Россия,Краснодар,n/a,15-20,36+,1,detractors
325212,QTT7IR,-13,new,NaN,other,IOS,SMARTPHONE,Россия,Москва,n/a,10-15,36+,7,passives
347784,RZUS6H,-6,new,NaN,other,ANDROID,SMARTPHONE,Россия,Красноярск,n/a,30-35,36+,3,detractors
385897,TYWQW4,-12,new,NaN,other,IOS,SMARTPHONE,Россия,Новосибирск,n/a,1-5,36+,5,detractors
401226,URKAFI,-11,new,NaN,other,ANDROID,SMARTPHONE,Россия,НижнийНовгород,n/a,10-15,36+,8,passives
407160,V2GNK2,-13,new,NaN,other,IOS,SMARTPHONE,Россия,РостовнаДону,n/a,15-20,36+,5,detractors


In [18]:
print('Отрицательные значения срока заключения контракта содержатся в', problem_df1.count()[0], 'строках.')

Отрицательные значения срока заключения контракта содержатся в 13 строках.


In [19]:
df['is_new'].unique()

array(['old', 'new'], dtype=object)

In [20]:
df['gender_segment'].unique()

array(['women', 'men', 'other'], dtype=object)

In [21]:
df['gender_segment'].value_counts()

gender_segment
women    272442
men      228750
other      1301
Name: count, dtype: int64

In [22]:
df['gender_segment'].value_counts()

gender_segment
women    272442
men      228750
other      1301
Name: count, dtype: int64

In [23]:
problem_df2=df.loc[df['gender_segment']=='other']
a = problem_df2.shape [0]
problem_df2.head(5)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
751,A1E59W,4055,old,NaN,other,ANDROID,SMARTPHONE,Россия,Екатеринбург,n/a,1-5,36+,1,detractors
950,A1S6VL,335,new,40.0,other,ANDROID,TABLET,Россия,Иваново,35-44,15-20,7-12,8,passives
1209,A28ZDT,4243,old,NaN,other,ANDROID,SMARTPHONE,Россия,Хабаровск,n/a,1-5,36+,5,detractors
1321,A2GLPQ,2354,old,NaN,other,ANDROID,SMARTPHONE,Россия,Иркутск,n/a,10-15,36+,1,detractors
2113,A3XMNW,41,new,32.0,other,ANDROID,SMARTPHONE,Россия,СанктПетербург,25-34,0.1-1,2,1,detractors


In [24]:
print ('Данные с проблемами в колонке gender_segment', a, 'строк.')

Данные с проблемами в колонке gender_segment 1301 строк.


In [25]:
df['city'].unique()

array(['Уфа', 'Киров', 'Москва', 'РостовнаДону', 'Рязань', 'Омск',
       'СанктПетербург', 'Волгоград', 'Тольятти', 'Казань', 'Самара',
       'Красноярск', 'Екатеринбург', 'Калуга', 'Краснодар', 'Иркутск',
       'Пермь', 'Владимир', 'Ижевск', 'Тюмень', 'Оренбург',
       'НижнийНовгород', 'Брянск', 'Челябинск', 'Астрахань', 'Сургут',
       'Тверь', 'Новосибирск', 'НабережныеЧелны', 'Махачкала', 'Воронеж',
       'Курск', 'Владивосток', 'Балашиха', 'Пенза', 'Калининград', 'Тула',
       'Саратов', 'Кемерово', 'Белгород', 'Барнаул', 'Чебоксары',
       'Архангельск', 'Томск', 'Ярославль', 'Ульяновск', 'Хабаровск',
       'Грозный', 'Ставрополь', 'Липецк', 'Новокузнецк', 'Якутск',
       'УланУдэ', 'Сочи', 'Иваново', 'НижнийТагил', 'Смоленск',
       'Волжский', 'Магнитогорск', 'Чита', 'Череповец', 'Саранск'],
      dtype=object)

In [26]:
df['nps_score'].unique() 

array([10,  6,  9,  2,  5,  8,  1,  4,  7,  3], dtype=int64)

In [27]:
df['age'].unique() 

array([45., 53., 57., 44., 24., 42., 35., 36., 54., 39., 21., 27., 60.,
       34., 47., 37., 43., 33., 31., 25., 51., 28., 41., 40., 46., 48.,
       32., 30., 52., 59., 26., 50., 62., 29., 55., 22., 38., 56., 23.,
       49., 66., 74., 75., 17., 65., 64., 69., 58., 20., 19., 80., 70.,
       81., 63., 67., 68., 72., 15., 79., 18., 73., nan, 14., 71., 61.,
       16., 77., 13., 76., 10., 78., 12., 82., 11., 83., 89., 84., 85.,
       87., 86.])

In [28]:
problem_df3=df.loc[df['age'].isna()]
b=problem_df3.shape [0]
problem_df3

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
751,A1E59W,4055,old,NaN,other,ANDROID,SMARTPHONE,Россия,Екатеринбург,n/a,1-5,36+,1,detractors
1209,A28ZDT,4243,old,NaN,other,ANDROID,SMARTPHONE,Россия,Хабаровск,n/a,1-5,36+,5,detractors
1321,A2GLPQ,2354,old,NaN,other,ANDROID,SMARTPHONE,Россия,Иркутск,n/a,10-15,36+,1,detractors
2163,A41C1K,557,old,NaN,other,ANDROID,SMARTPHONE,Россия,РостовнаДону,n/a,1-5,13-24,3,detractors
2667,A513VG,4705,old,NaN,other,IOS,SMARTPHONE,Россия,Екатеринбург,n/a,95-100,36+,9,promoters
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499598,ZUDYU9,520,old,NaN,other,ANDROID,SMARTPHONE,Россия,СанктПетербург,n/a,100+,13-24,9,promoters
500834,ZWQJRW,220,new,NaN,men,ANDROID,SMARTPHONE,Россия,Самара,n/a,0.1-1,7-12,1,detractors
501246,ZXK9AK,3550,old,NaN,other,ANDROID,SMARTPHONE,Россия,Томск,n/a,1-5,36+,4,detractors
502376,ZZRS2G,345,new,NaN,men,ANDROID,SMARTPHONE,Россия,Москва,n/a,1-5,7-12,5,detractors


In [29]:
problem_df=(df.loc[(df['age'].isna()) & (df['gender_segment']=='other')])
problem_df

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
751,A1E59W,4055,old,NaN,other,ANDROID,SMARTPHONE,Россия,Екатеринбург,n/a,1-5,36+,1,detractors
1209,A28ZDT,4243,old,NaN,other,ANDROID,SMARTPHONE,Россия,Хабаровск,n/a,1-5,36+,5,detractors
1321,A2GLPQ,2354,old,NaN,other,ANDROID,SMARTPHONE,Россия,Иркутск,n/a,10-15,36+,1,detractors
2163,A41C1K,557,old,NaN,other,ANDROID,SMARTPHONE,Россия,РостовнаДону,n/a,1-5,13-24,3,detractors
2667,A513VG,4705,old,NaN,other,IOS,SMARTPHONE,Россия,Екатеринбург,n/a,95-100,36+,9,promoters
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498652,ZSL1TE,1389,old,NaN,other,ANDROID,SMARTPHONE,Россия,Калининград,n/a,1-5,36+,10,promoters
498998,ZT6Z5A,1295,old,NaN,other,ANDROID,SMARTPHONE,Россия,Рязань,n/a,1-5,36+,5,detractors
499598,ZUDYU9,520,old,NaN,other,ANDROID,SMARTPHONE,Россия,СанктПетербург,n/a,100+,13-24,9,promoters
501246,ZXK9AK,3550,old,NaN,other,ANDROID,SMARTPHONE,Россия,Томск,n/a,1-5,36+,4,detractors


In [30]:
notproblem_df=(df.loc[(df['age'].isna()) & (df['gender_segment']!='other')])
notproblem_df.head (5)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
24321,B9G5PC,949,old,NaN,men,ANDROID,SMARTPHONE,Россия,Москва,n/a,40-45,25-36,1,detractors
33612,BQNYFS,4,new,NaN,women,IOS,SMARTPHONE,Россия,Киров,n/a,1-5,1,10,promoters
38525,BZQ2EP,472,old,NaN,women,ANDROID,SMARTPHONE,Россия,Новосибирск,n/a,1-5,13-24,8,passives
42431,C6U342,2735,old,NaN,men,ANDROID,SMARTPHONE,Россия,Краснодар,n/a,10-15,36+,1,detractors
52158,CONZ1N,832,old,NaN,men,ANDROID,SMARTPHONE,Россия,Оренбург,n/a,85-90,25-36,2,detractors


**Вывод**

В ходе данного этапа работы с помощью запроса SQL были обьеденены имеющиеся таблицы.
В результате мы получили данные соедржащие информацию о 502493 пользователях.
В данных есть колонка lt_day сщ значением менее 0 в 13 случаях. Следует отметить, что в этих 13 строках также отсутствует пол и возраст. 
Также в данных есть 554 польователя с отсутсвующей информацией по колонке age и age_segment в 514 случаях в этих данных указан пол другой. 
Также есть данные где в качестве пола указано 'other' в кол-ве 1301 штука.
Таким образом удалив 1301 строку с отсутствием пола мы избавимся от отрицательных значений по колонке с днями заключения конракта и 514 строк с незаполненным возрастом. 
Данная обработка будет осуществляться в программе tableau будет применены соответсвующие фильтры на уровне источника данных

## 5. Ссылка на дашборд <a id="start5"></a> 

ссылка на интерактивный дашборд

https://public.tableau.com/shared/5392TJYKC?:display_count=n&:origin=viz_share_link